<a href="https://colab.research.google.com/github/olaakintola/WheelSpace/blob/main/busentiredata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ENTIRE BUS DATA, TRAIN TEST SPLIT - OVERSAMMPLING

# The entire dataset was read into colab and split using train-test-split function
# into training and test datasets. Oversampling was used to improve the imbalanced
# classification. Random Forest classifier was used here.

!pip install -q sklearn

In [2]:
%tensorflow_version 2.x

In [3]:
from __future__ import absolute_import, division, print_function,unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
bus_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/busdata.csv")

In [5]:
bus_df["wheeloccupancy"]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
145436    0.0
145437    0.0
145438    0.0
145439    0.0
145440    NaN
Name: wheeloccupancy, Length: 145441, dtype: float64

In [6]:
bus_df["wheeloccupancy"] = bus_df["wheeloccupancy"].fillna(0).astype(int)

In [7]:
bus_df["wheeloccupancy"]

0         0
1         0
2         0
3         0
4         0
         ..
145436    0
145437    0
145438    0
145439    0
145440    0
Name: wheeloccupancy, Length: 145441, dtype: int64

In [8]:
bus_df.shape

(145441, 6)

In [9]:
bus_df.sample(5)

,route,times,days,wheeloccupancy,departure,destination
17053,68A,08:00,tues,0,NaN,NaN
110602,25X,17:00,tues,0,NaN,NaN
24226,9,17:00,thurs,0,NaN,NaN
80635,27X,14:00,fri,1,"""The Link Road, stop 1259""","""Marino College, stop 523"""
55894,25,17:00,wed,0,NaN,NaN


In [10]:
bus_df.fillna('', inplace=True)

In [11]:
bus_df.sample(5)

,route,times,days,wheeloccupancy,departure,destination
117395,33,18:00,wed,0,,
24685,13,08:00,wed,0,,
46578,54A,13:00,tues,0,,
97505,14,12:00,mon,0,,
42216,84,07:00,thurs,0,,


In [12]:
bus_df[bus_df.wheeloccupancy==1]

,route,times,days,wheeloccupancy,departure,destination
86,11,09:00,wed,1,"""Drummartin Park, stop 3024""","""Parnell Square West, stop 10"""
126,116,13:00,mon,1,"""Teresian School, stop 771""","""Stillorgan College, stop 2011"""
134,116,09:00,tues,1,"""Stillorgan Luas, stop 7057""","""Carmanhall Road, stop 3181"""
182,118,09:00,mon,1,"""St Johns Ambulance, stop 907""","""Donnybrook Garda Stn, stop 774"""
243,120,10:00,mon,1,"""Naomh Fionnbarra GAA, stop 4485""","""St Finbar's Road, stop 829"""
...,...,...,...,...,...,...
145264,84A,11:00,mon,1,"""St John's Church, stop 3080""","""Old Connaught Avenue, stop 4129"""
145275,84A,10:00,tues,1,"""Castle Street, stop 4130""","""Merrion Gates, stop 476"""
145299,84A,10:00,thurs,1,"""St James's Church, stop 4203""","""Shrewsbury Lawn, stop 3130"""
145324,84X,11:00,mon,1,"""St Anthony Church, stop 4268""","""Greystones Station, stop 4283"""


In [13]:
bus_df["departure"] = [x.replace("'","").replace('"','') for x in bus_df["departure"]]

In [14]:
bus_df["destination"] = [x.replace("'","").replace('"','') for x in bus_df["destination"]]

In [15]:
bus_df.drop("departure", axis="columns", inplace=True)

In [16]:
bus_df.drop("destination", axis="columns", inplace=True)

In [17]:
bus_df.dtypes

route             object
times             object
days              object
wheeloccupancy     int64
dtype: object

In [18]:
bus_df.head()

,route,times,days,wheeloccupancy
0,1,07:00,mon,0
1,1,08:00,mon,0
2,1,09:00,mon,0
3,1,10:00,mon,0
4,1,11:00,mon,0


In [19]:
def print_unique_col_values(bus_df):
  for column in bus_df:
    if bus_df[column].dtypes=='object':
      print(bus_df[column].unique())

In [20]:
print_unique_col_values(bus_df)

['1' '11' '116' '118' '120' '122' '123' '13' '130' '14' '140' '142' '145'
 '15' '150' '151' '155' '15A' '15B' '15D' '16' '16D' '25' '25A' '25B'
 '25X' '26' '27' '27A' '27B' '27X' '29A' '31' '31A' '31B' '32' '32X' '33'
 '33D' '33E' '33X' '37' '38' '38A' '38B' '38D' '39' '39A' '4' '40' '40B'
 '40D' '40E' '41' '41B' '41C' '41D' '41X' '42' '43' '44' '44B' '46A' '46E'
 '47' '49' '51D' '51X' '53' '54A' '56A' '61' '65' '65B' '66' '66A' '66B'
 '66E' '66X' '67' '67X' '68' '68A' '68X' '69' '69X' '7' '70' '77A' '77X'
 '79' '79A' '7A' '7B' '7D' '83' '83A' '84' '84A' '84X' '9' '\x1a']
['07:00' '08:00' '09:00' '10:00' '11:00' '12:00' '13:00' '14:00' '15:00'
 '16:00' '17:00' '18:00' '']
['mon' 'tues' 'wed' 'thurs' 'fri' '']


In [21]:
X_over_bus = bus_df.drop("wheeloccupancy", axis="columns") #oversampling
y_over_bus = bus_df["wheeloccupancy"]

from sklearn.model_selection import train_test_split
X_train_bus, X_test_bus, y_train_bus, y_test_bus = train_test_split(X_over_bus, y_over_bus, test_size=0.2, random_state=15)

In [22]:
## beginning of oversampling for all routes - bus_df

#class count
count_class_bus_0, count_class_bus_1 = y_train_bus.value_counts()

#Divide by class
df_class_bus_0 = X_train_bus[y_train_bus==0]
df_class_bus_1 = X_train_bus[y_train_bus==1]

In [23]:
count_class_bus_0, count_class_bus_1

(110962, 5390)

In [24]:
df_class_bus_0.shape

(110962, 3)

In [26]:
df_class_bus_1.shape

(5390, 3)

In [27]:
df_over_1_bus = df_class_bus_1.sample(count_class_bus_0, replace=True) #oversample

X_oversampling_bus = pd.concat([df_class_bus_0, df_over_1_bus], axis=0)

y_oversampling_bus =  np.concatenate([np.zeros(len(df_class_bus_0 ) ), np.ones(len(df_over_1_bus) )])

In [28]:

y_oversampling_bus.shape#

(221924,)

In [29]:
X_oversampling_bus.shape

(221924, 3)

In [30]:
y_train_bus.value_counts()

0    110962
1      5390
Name: wheeloccupancy, dtype: int64

In [31]:
from sklearn.preprocessing import OneHotEncoder
ohe_all_route=OneHotEncoder(handle_unknown='ignore')
X_train_hot_all=ohe_all_route.fit_transform(X_oversampling_bus)
X_test_hot_all=ohe_all_route.transform(X_test_bus)

In [32]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
#clf.fit(trainingdata_encode_shuffle, train_label_shuffle)
#print(clf.predict([[0, 0, 0, 0]]))
#my second version
#clf.fit(trainingdata_encode, train_label)

#bus_df_encode variables
clf.fit(X_train_hot_all, y_oversampling_bus)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [33]:
preds_hotall_route = clf.predict(X_test_hot_all) 

In [34]:
preds_hotall_route[:10]

array([1., 1., 1., 1., 1., 0., 0., 0., 1., 0.])

In [35]:
y_test_bus[:10]

123533    0
45355     0
143012    0
128420    0
57176     0
134578    0
87178     0
62905     0
36376     0
74109     0
Name: wheeloccupancy, dtype: int64

In [36]:
from sklearn.metrics import  confusion_matrix, classification_report

print(classification_report (y_test_bus, preds_hotall_route) )

              precision    recall  f1-score   support

           0       0.99      0.43      0.60     27695
           1       0.07      0.88      0.13      1394

    accuracy                           0.45     29089
   macro avg       0.53      0.66      0.37     29089
weighted avg       0.94      0.45      0.58     29089

